# accuracy比較（TF-IDF vs Doc2Vec）

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) TF-IDF版のaccuracyを計測

### (2-1) vectorizerを生成（my-ope現行仕様に従う）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 13
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
question_answers = _datasource.question_answers(_bot_id)
questions = np.array(question_answers['question'])
answer_ids = np.array(question_answers['answer_id'])
ids = np.array(question_answers['id'])

In [4]:
from learning.core.training_set.text_array import TextArray

all_body_array = TextArray(questions)
_ = all_body_array.to_vec()
vectorizer_for_accuracy = all_body_array.vectorizer

### (2-2) accuracy計測用のセットを作成

学習セットから「」で囲まれた部分を削除しておきます

In [5]:
import re

question_start_pattern = r'(^「[^「]+」)(.+)'
questions_for_accuracy = []
for _q in questions:
    # 改行文字、全角スペースが含まれていることがある
    _q = _q.replace('\n', '')
    _q = _q.replace(u'　', ' ')
    
    m = re.match(question_start_pattern, _q)
    try:
        '''
        「」で囲まれた部分以外を
        accuracy計測用の質問文として取得
        
        ただしid=13600「こんにちは」は「」がない
        '''
        if _q == 'こんにちは':
            question = 'こんにちは。'
        else:
            question = m.group(2)
    except:
        question = None
        pass

    questions_for_accuracy.append(question)

len(questions_for_accuracy)

317

#### 長いですが、確認のため全量を表示します。

In [6]:
questions_for_accuracy

['申請内容を修正したいので、差戻しをお願いしたい',
 '当社と住宅ローンで提携している銀行名、優遇条件、連絡窓口等を知りたい',
 '保育園入所に伴う役所へ提出するため、証明書（勤務/育児休業）が欲しい',
 '在職証明書が欲しい',
 '不動産法人割引の紹介カードが欲しい。',
 '新規募集時期はいつですか？',
 '金額の変更は随時できますか？',
 '払戻しをお願いしたい。',
 '社宅はどこにありますか？空き状況は？',
 '出張者から金町寮に宿泊したいがどうしたらいいか。',
 '引っ越し先が決まっていないが、どうしたらいいか。',
 '港区に勤務している在勤証明書を発行してほしい。',
 '人事異動（出向戻り）に伴うPCアドレスの設定はどうやるの？',
 '育児短時間勤務申請書はいつまでに提出するのか？',
 '育児短時間勤務申請書は曜日別に申請できるか？',
 '育児短時間勤務申請はどこへ提出すればよいか？',
 '一部解約をしたいが、申込みから振込までどれくらいかかりますか？',
 '現在、借上げ寮にいますが、子供が大学に入学し上京するため、一緒に住める寮に変更したい。',
 'コーポレートカードを持っているか解らなくなったので、調べてほしい',
 '芝浦アイランド付近に住んでいるが、都バスで利用（申請）してもよいか？',
 '最寄駅までバスの走行距離が1ｋｍ以上あるのに、1ｋｍ未満で不支給になったのはなぜ？',
 '東京本社から近いので、バスのみで通勤申請してもよいか？',
 '経済的・合理的なルートとは？',
 '月10万円を超えなければ、新幹線通勤は認めてくれるの？',
 '送付制度を利用した際、現地発生の関税費用等を立替えているが精算は？',
 '現地住所が決まっていないが、引越し手配は進められるか？',
 '勤務証明やお子様の駐在証明を発行できますか？',
 '予防接種の費用負担先はどこですか？勘定科目は？',
 '休暇申請の申請内容を変更したいのですが。再申請した方がいいですか？',
 '予約を変更したのですが、どうしたらいいですか？',
 '健保での健診を半年以内に受診してますが、渡航前健診として代用できますか？',
 '中国指定の健診を受診したら、渡航前健診は受けなくてもいいですか？',
 '駐在時に預かった保険証を、返却してくだ

### (2-3) accuracy計測用の予測処理

In [7]:
'''
    プロダクションで使用しているクラスから抽出
'''
from sklearn.metrics.pairwise import cosine_similarity
from learning.core.datasource import Datasource
from learning.core.training_set.text_array import TextArray
from learning.log import logger

def question_answers(question, datasource_type='csv'):
    """質問文間でコサイン類似度を算出して、近い質問文の候補を取得する
    """
    datasource = Datasource(type=datasource_type)
    question_answers = datasource.question_answers_for_suggest(_bot_id, question)
    all_array = TextArray(question_answers['question'], vectorizer=vectorizer_for_accuracy)
    question_array = TextArray([question], vectorizer=vectorizer_for_accuracy)

    similarities = cosine_similarity(all_array.to_vec(), question_array.to_vec())
    similarities = similarities.flatten()
    #logger.debug("similarities: %s" % similarities)

    ordered_result = list(map(lambda x: {
        'question_answer_id': float(x[0]), 'similarity': x[1]
    }, sorted(zip(question_answers['id'], similarities), key=lambda x: x[1], reverse=True)))

    ordered_result = list(filter((lambda x: x['similarity'] > 0.1), ordered_result))

    # 先頭要素だけを戻す
    return ordered_result[0]

### (2-4) accuracy計測

In [8]:
total_cnt = len(questions_for_accuracy)
accurate_cnt = 0
for index in range(total_cnt):
    question = questions_for_accuracy[index]
    question_answer_id = ids[index]

    result = question_answers(question)
    if result['question_answer_id'] == question_answer_id and result['similarity'] > 0.49:
        accurate_cnt += 1
    else:
        print('不正解：予測対象＝%d --> 結果＝%d (similarity=%0.3f)' % (
            question_answer_id, result['question_answer_id'], result['similarity']
        ))

不正解：予測対象＝13279 --> 結果＝13506 (similarity=0.952)
不正解：予測対象＝13290 --> 結果＝13522 (similarity=0.952)
不正解：予測対象＝13309 --> 結果＝13324 (similarity=0.954)
不正解：予測対象＝13315 --> 結果＝13304 (similarity=0.740)
不正解：予測対象＝13321 --> 結果＝13311 (similarity=0.767)
不正解：予測対象＝13336 --> 結果＝13402 (similarity=0.954)
不正解：予測対象＝13527 --> 結果＝13527 (similarity=0.486)
不正解：予測対象＝13528 --> 結果＝13527 (similarity=0.486)


In [9]:
print('accuracy=%0.3f' % (accurate_cnt/total_cnt))

accuracy=0.975


## (3) Doc2Vec版のaccuracyを計測

### (3-1) モデルをロード

レポート <a href="25-Create-doc2vec-model-wiki.ipynb"><b>25-Create-doc2vec-model-wiki.ipynb</b></a> の手順にて生成したDoc2Vecモデルファイルをロードして使用します。

上記手順では、Wikipedia文書、my-ope文書の双方を使用し、ボキャブラリ／単語ベクトルの生成および学習を行い、モデルをファイル保存しています。

In [10]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wiki_myope.PV%d.model' % dm
    return model_path

In [11]:
'''
    あらかじめ学習したモデルのファイルをロード
    dm = 0 : DBoWを使用したモデル
'''
dm = 0
loaded_model_dbow = models.Doc2Vec.load(doc2vec_model_path(dm))

print('Document vector size=%d' % (len(loaded_model_dbow.docvecs)))

Document vector size=87349


### (3-2) accuracy計測用の予測処理

レポート <a href="26-Doc2Vec-Wiki-with-scikit_learn.ipynb"><b>26-Doc2Vec-Wiki-with-scikit_learn.ipynb</b></a> にて使用したものと同じものになります。

In [12]:
from learning.core.nlang import Nlang

def get_document_vector(question, model, warning):
    '''
        question: 
            分かち書きされていない文書
        model:
            Doc2Vecの学習済みモデル
            （検証時は品詞を落としていないWikipedia文書からモデルを生成）

        inferred_vector:
            文書を分かち書きしたコーパスから、
            Doc2Vecの学習済みモデルを使用して
            生成される類似文書ベクトル
            （learning.core.nlang.Nlangの仕様に従い、
            　一部品詞が落とされます。）

        warning:
            Trueを指定時、コーパスに含まれる単語が
            モデル内のWord2Vecボキャブラリにない場合、
            警告を表示する
    '''
    corpus = Nlang.split(question).split()
    inferred_vector = model.infer_vector(corpus, alpha=0.01, min_alpha=0.001, steps=250)
    
    if warning:
        for c in corpus:
            if not c in model.wv.vocab:
                print("Warning: word [%s] does not exist in Word2Vec vocabulary." % c)

    return inferred_vector

def get_document_vectors(questions, model, warning=False):
    document_vectors = []
    for question in questions:
        inferred_vector = get_document_vector(question, model, warning)
        document_vectors.append(list(inferred_vector))

    return np.array(document_vectors)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from learning.core.datasource import Datasource
import time

def search_simiarity_doc2vec(question, dbow_model):
    '''
        質問文間でコサイン類似度を算出して、近い質問文の候補を取得する
        
        仕様はプロダクションに準拠しています
        ただし、文書のベクトル化は、TF-IDFではなく、
        Doc2Vecを使用します。
    '''
    datasource = Datasource('csv')
    question_answers = datasource.question_answers_for_suggest(_bot_id, question)

    #all_array = TextArray(question_answers['question'], vectorizer=self.vectorizer)
    #question_array = TextArray([question], vectorizer=self.vectorizer)
    all_array      = get_document_vectors(question_answers['question'], dbow_model)
    question_array = get_document_vectors([question], dbow_model, warning=True)
    
    similarities = cosine_similarity(all_array, question_array)
    similarities = similarities.flatten()

    ordered_result = list(map(lambda x: {
        'question_answer_id': float(x[0]), 'similarity': x[1], 'answer_id': x[2]
    }, sorted(zip(question_answers['id'], similarities, question_answers['answer_id']), key=lambda x: x[1], reverse=True)))

    ordered_result = list(filter((lambda x: x['similarity'] > 0.1), ordered_result))

    # 先頭要素だけを戻す
    return ordered_result[0]

### (3-3) accuracy計測

In [14]:
total_cnt = len(questions_for_accuracy)
accurate_cnt = 0
for index in range(total_cnt):
    question = questions_for_accuracy[index]
    question_answer_id = ids[index]

    result = search_simiarity_doc2vec(question, loaded_model_dbow)
    if result['question_answer_id'] == question_answer_id and result['similarity'] > 0.49:
        accurate_cnt += 1
    else:
        print('不正解：予測対象＝%d --> 結果＝%d (similarity=%0.3f)' % (
            question_answer_id, result['question_answer_id'], result['similarity']
        ))

不正解：予測対象＝13279 --> 結果＝13506 (similarity=0.990)
不正解：予測対象＝13309 --> 結果＝13324 (similarity=0.967)
不正解：予測対象＝13315 --> 結果＝13304 (similarity=0.924)
不正解：予測対象＝13321 --> 結果＝13311 (similarity=0.917)
不正解：予測対象＝13402 --> 結果＝13336 (similarity=0.992)
不正解：予測対象＝13522 --> 結果＝13290 (similarity=0.991)
不正解：予測対象＝13528 --> 結果＝13527 (similarity=0.770)


In [15]:
print('accuracy=%0.3f' % (accurate_cnt/total_cnt))

accuracy=0.978
